In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
anime = pd.read_csv('../anime-recommendations-database/anime.csv')
animeRating = pd.read_csv('../anime-recommendations-database/rating.csv')

### Cleaning Data for Missing Values

In [3]:
#Checking which values are null
print anime.isnull().sum()
print animeRating.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64
user_id     0
anime_id    0
rating      0
dtype: int64


In [4]:
anime['genre'] = anime['genre'].fillna('None')
anime['type'] = anime['type'].fillna('None')
anime['rating'] = anime['rating'].fillna('None')
anime.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [5]:
animeRating = animeRating[animeRating.rating > 0]
animeRating.rating.unique()

array([10,  8,  6,  9,  7,  3,  5,  4,  1,  2], dtype=int64)

In [6]:
fullMergedAnime = animeRating.merge(anime, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
fullMergedAnime.head()

,user_id,anime_id,rating_user,name,genre,type,episodes,rating,members
0,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
1,3,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
2,5,8074,2,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
3,12,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
4,14,8074,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892


### Splitting and Normalizing Data

In [7]:
trainData, testData = train_test_split(fullMergedAnime, test_size=0.2)

In [8]:
subTrainData = trainData[['user_id', 'anime_id', 'name', 'rating_user']]
#Limiting the dataframe
subTrainData = subTrainData[subTrainData.user_id <= 6000]

In [9]:
pivTrain = subTrainData.pivot_table(index=['user_id'], columns=['name'], values='rating_user')
print(pivTrain.shape)
pivTrain.head()

(5606, 6780)


name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
subTestData = testData[['user_id', 'anime_id', 'name', 'rating_user']]
#Limiting the dataframe
subTestData = subTestData[subTestData.user_id <= 6000]

In [11]:
pivTest = subTestData.pivot_table(index=['user_id'], columns=['name'], values='rating_user')
print(pivTest.shape)
pivTest.head()

(5069, 5136)


name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def create_Normalized_Matrix(piv):
    piv = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
    piv.fillna(0, inplace=True)
    piv = piv.T
    piv = piv.loc[:, (piv != 0).any(axis=0)]
    return piv

In [13]:
pivTrainNorm = create_Normalized_Matrix(pivTrain)
print (pivTrainNorm.shape)

(6780, 5120)


In [14]:
pivTestNorm = create_Normalized_Matrix(pivTest)
print (pivTestNorm.shape)

(5136, 4395)


### Create Sparse Matrices

In [15]:
def create_Sparse_Matrix(userAnimeMatrix):
    return sp.sparse.csr_matrix(userAnimeMatrix.values)

In [16]:
pivTrainSparse = create_Sparse_Matrix(pivTrainNorm)

### Item-Item Cosine Similarity

In [17]:
def item_Cosine_Similarity(itemMatrix):
    return cosine_similarity(itemMatrix)

def item_Similarity_DataFrame(itemSim, pivNorm):
    return pd.DataFrame(itemSim, index = pivNorm.index, columns = pivNorm.index)

In [18]:
itemCosineSim = item_Cosine_Similarity(pivTrainSparse)
trainItemSimData = item_Similarity_DataFrame(itemCosineSim, pivTrainNorm)
trainItemSimData.head()

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,1.000000,0.161675,0.215310,-0.001366,0.038945,-0.131438,-0.044528,0.000123,-0.062613,0.004168,...,0.0,0.000000,0.013971,0.000000,0.057788,0.064614,0.114775,0.100432,0.108838,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.161675,1.000000,0.263576,-0.017160,0.028893,-0.249021,-0.105970,-0.011278,-0.095333,0.010084,...,0.0,0.000345,0.000000,-0.052951,-0.027997,0.037681,-0.030469,-0.066979,-0.078972,0.0
&quot;Bungaku Shoujo&quot; Movie,0.215310,0.263576,1.000000,0.002162,0.041009,-0.151323,-0.089945,-0.010636,-0.085264,0.025262,...,0.0,0.000000,-0.013879,0.000000,-0.008144,0.021686,0.041121,0.035961,0.025136,0.0
.hack//G.U. Returner,-0.001366,-0.017160,0.002162,1.000000,0.179289,0.092026,0.086559,0.418163,0.243162,0.088149,...,0.0,0.000000,0.000000,0.000000,-0.078149,-0.066525,0.028175,-0.087675,-0.154139,0.0
.hack//G.U. Trilogy,0.038945,0.028893,0.041009,0.179289,1.000000,0.061632,0.087967,0.104654,0.004676,0.004214,...,0.0,0.000000,0.010202,0.013413,-0.011520,0.006183,0.044456,0.053164,0.024037,0.0


### Item-Item Cosine Implementation

In [19]:
def items_Avg_Rating_Matrix(itemMatrix):
    avgItemsRating = {}
    for itemID in itemMatrix:
        ratingsSum = 0.0
        counter = 0
        for rating in itemMatrix[itemID]:
            if not np.isnan(rating):
                ratingsSum += rating
                counter += 1
            else:
                continue
        avgItemsRating[itemID] = ratingsSum / counter
    return avgItemsRating

def item_Avg_Rating(itemID):
    itemRatings = pivTrain.loc[itemID, :]
    avgItemRatingList = 0.0
    counter = 0
    for rating in itemRatings:
        if not np.isnan(rating):
            avgItemRatingList += rating
            counter += 1
        else:
            continue
    return avgItemRatingList/counter

In [20]:
avgItemRatings = items_Avg_Rating_Matrix(pivTrain)
# print avgItemRatings
# avgItemRating = item_Avg_Rating(3)
# print avgItemRating

In [21]:
def rating_Item_Cosine_Prediction(userID, animeName):
    if userID in pivTrainNorm.columns:
        allItemSimilarity = trainItemSimData.sort_values(by=animeName, ascending=False).loc[:,animeName]
        itemAvg = avgItemRatings[animeName]
        ratingWeight = 0.0
        similarityWeight = 0.0
        topNUsers = 0
        
        for itemX in allItemSimilarity.index:
            if animeName == itemX:
                continue
            
            itemXRating = pivTrain.loc[userID, itemX]
            if itemXRating < 0.01:
                continue
    
            if topNUsers < 50:
                itemXAvg = avgItemRatings[itemX]
                itemXSimilarity = allItemSimilarity.loc[itemX]
                if np.isnan(itemXRating):
                    continue
                elif not np.isnan(itemXRating):
                    ratingWeight += (itemXRating - itemXAvg)*itemXSimilarity
                    similarityWeight += abs(itemXSimilarity)
                    topNUsers += 1

    else:
        return ("No data available for UserID: {}".format(userID))
    
    return itemAvg + (ratingWeight / similarityWeight)

In [22]:
itemCosinePrediction = rating_Item_Cosine_Prediction(3, "Zombie-Loan")
print itemCosinePrediction

6.693252992288715


### Evaluation of Item-Item Cosine Similarity using MAE and RMSE

In [23]:
# Mean Absolute Error and Root Mean Squared Error
import math
def calc_Cosine_Error():
    meanAbsErrorList = 0.0
    rootMSEList = 0.0
    count = 0
    for anime in pivTest.columns:
        if anime in pivTestNorm.index:
            itemInfo = pivTest.loc[:, anime]
            for userID in itemInfo.index:
                trueRating = itemInfo[userID]
                if np.isnan(trueRating):
                    continue
                predictedRating = rating_Item_Cosine_Prediction(userID, anime)
                meanAbsErrorList += abs(predictedRating - trueRating)
                rootMSEList += (predictedRating - trueRating)**2
                count += 1
    return meanAbsErrorList/count, math.sqrt(rootMSEList/count)

In [76]:
cosineMAE, cosineRMSE = calc_Cosine_Error()
print "MAE: {}".format(cosineMAE)
print "RMSE: {}".format(cosineRMSE)

1.0006945928569715


### Item-Item Pearson Similarity

In [26]:
def pearson_Covariance(itemMatrix):
    return np.cov(itemMatrix)

def pearson_Similarity(itemCovariance):
    return np.corrcoef(itemCovariance)

In [27]:
itemPearsonCovariance = pearson_Covariance(pivTrainNorm.values)

In [28]:
itemPearsonSim = pearson_Similarity(itemPearsonCovariance)

C:\Users\Hernandez-TAMU\Anaconda2\envs\cs670\lib\site-packages\numpy\lib\function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Hernandez-TAMU\Anaconda2\envs\cs670\lib\site-packages\numpy\lib\function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [29]:
trainItemPSimData = item_Similarity_DataFrame(itemPearsonSim, pivTrainNorm)
trainItemPSimData.head()

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,1.000000,0.098315,0.299798,-0.050094,0.074303,-0.136677,-0.064022,-0.056066,-0.100450,0.038155,...,-0.009234,-0.038737,-0.006869,0.019452,0.201674,0.183179,0.314348,0.266959,0.255924,0.030515
&quot;Bungaku Shoujo&quot; Memoire,0.098315,1.000000,0.321392,-0.025891,0.071005,-0.239784,-0.130967,-0.018206,-0.140926,0.011126,...,-0.007050,-0.006382,-0.011489,-0.050795,-0.092922,0.058097,-0.125238,-0.136172,-0.149680,0.023176
&quot;Bungaku Shoujo&quot; Movie,0.299798,0.321392,1.000000,-0.176810,0.035272,-0.267096,-0.255708,-0.206954,-0.263878,0.015581,...,0.084531,-0.135897,-0.075895,0.002266,0.076769,0.170040,0.251650,0.264401,0.288801,-0.100427
.hack//G.U. Returner,-0.050094,-0.025891,-0.176810,1.000000,0.225632,0.207223,0.247240,0.832848,0.392866,0.158088,...,-0.057279,0.073108,0.059142,0.012618,-0.319545,-0.399158,-0.195558,-0.463388,-0.560174,0.125831
.hack//G.U. Trilogy,0.074303,0.071005,0.035272,0.225632,1.000000,0.181058,0.162248,0.222114,0.080584,0.083679,...,-0.060506,0.013142,0.063381,0.011210,-0.071299,-0.063988,-0.019761,-0.029549,-0.099989,0.040984


### Item-Item Pearson Implementation

In [30]:
def items_Avg_Rating_Matrix(itemMatrix):
    avgItemsRating = {}
    for itemID in itemMatrix:
        ratingsSum = 0.0
        counter = 0
        for rating in itemMatrix[itemID]:
            if not np.isnan(rating):
                ratingsSum += rating
                counter += 1
            else:
                continue
        avgItemsRating[itemID] = ratingsSum / counter
    return avgItemsRating

def item_Avg_Rating(itemID):
    itemRatings = pivTrain.loc[itemID, :]
    avgItemRatingList = 0.0
    counter = 0
    for rating in itemRatings:
        if not np.isnan(rating):
            avgItemRatingList += rating
            counter += 1
        else:
            continue
    return avgItemRatingList/counter

In [31]:
avgItemRatings = items_Avg_Rating_Matrix(pivTrain)
# print avgItemRatings
# avgItemRating = item_Avg_Rating(3)
# print avgItemRating

In [32]:
def rating_Item_Pearson_Prediction(userID, animeName):
    if userID in pivTrainNorm.columns:
        allItemSimilarity = trainItemPSimData.sort_values(by=animeName, ascending=False).loc[:,animeName]
        itemAvg = avgItemRatings[animeName]
        ratingWeight = 0.0
        similarityWeight = 0.0
        topNUsers = 0
        
        for itemX in allItemSimilarity.index:
            if animeName == itemX:
                continue
            
            itemXRating = pivTrain.loc[userID, itemX]
            if itemXRating < 0.01:
                continue
    
            if topNUsers < 50:
                itemXAvg = avgItemRatings[itemX]
                itemXSimilarity = allItemSimilarity.loc[itemX]
                if np.isnan(itemXRating):
                    continue
                elif not np.isnan(itemXRating):
                    ratingWeight += (itemXRating - itemXAvg)*itemXSimilarity
                    similarityWeight += abs(itemXSimilarity)
                    topNUsers += 1

    else:
        return ("No data available for UserID: {}".format(userID))
    
    return itemAvg + (ratingWeight / similarityWeight)

In [33]:
itemPearsonPrediction = rating_Item_Pearson_Prediction(3, "Zombie-Loan")
print itemPearsonPrediction

6.680345096714367


### Evaluation of Item-Item Pearson Similarity using MAE and RMSE

In [34]:
# Mean Absolute Error and Root Mean Squared Error
import math
def calc_Pearson_Error():
    meanAbsErrorList = 0.0
    rootMSEList = 0.0
    count = 0
    for anime in pivTest.columns:
        if anime in pivTestNorm.index:
            itemInfo = pivTest.loc[:, anime]
            for userID in itemInfo.index:
                trueRating = itemInfo[userID]
                if np.isnan(trueRating):
                    continue
                predictedRating = rating_Item_Pearson_Prediction(userID, anime)
                meanAbsErrorList += abs(predictedRating - trueRating)
                rootMSEList += (predictedRating - trueRating)**2
                count += 1
    return meanAbsErrorList/count, math.sqrt(rootMSEList/count)

In [ ]:
pearsonMAE, pearsonRMSE = calc_Pearson_Error()
print "MAE: {}".format(pearsonMAE)
print "RMSE: {}".format(pearsonRMSE)